# 概要

方形波を対象としてフーリエ級数展開を試してみる．<br/>
準備として，次のコマンドを実行して関連ファイルをダウンロードしておいてください．

In [ ]:
!git clone https://github.com/knakamura1982/signal_processing.git

# 方形波の定義

一口に方形波と言っても様々であり，偶信号型のもの，奇信号型のもの，高周波なもの，低周波なものなど，色々ある．<br/>
ここでは，ひとまず次式で与えられる信号として定義しておく．
$$
{\rm SquareWave}(t) = \sum_{m=-\infty}^\infty p(t-mT) \ , \quad ただし \quad p(t)=\left\{
  \begin{array}{ll}
  1 & (0 \le t < \frac{T}{2}) \\
  0 & (t<0,\ \frac{T}{2} \le t)
  \end{array}
\right.
$$
ただし，上の定義は無限和を含むためコンピュータ上では扱えない．<br/>
そこで，$t$ を $T$ で割った余りを $d=t \mod T$ とし，次式のように表すことにする．
$$
{\rm SquareWave}(t) = \left\{
  \begin{array}{ll}
  1 & (0 \le d < \frac{T}{2}) \\
  0 & (\frac{T}{2} \le d < T)
  \end{array}
\right.
$$
これは，最初に定義した式と同じ意味になる．

実際に ${\rm SquareWave}(t)$ を生成してみる（$T=1$ とする）．

In [ ]:
import numpy as np
from signal_processing.utils import show_continuous_signal


### 設定項目：ここから ###

# 方形波を関数として定義
def square_wave(t):

  T = 1  # T を設定
  d = t % T  # t を T で割った余り

  '''
  if d < T/2:
    return 1
  else:
    return 0
  '''
  return np.where(d < T/2, 1, 0)  # 上の if 文に相当

# グラフ表示する範囲を指定
t_min = -3
t_max = 3

### 設定項目：ここまで ###


# 関数 square_wave を連続時間信号と解釈し，t = -3～3 [秒] の範囲をグラフ表示
show_continuous_signal(square_wave, trange=[t_min, t_max], vrange=[-1.5, 2.5], title='square wave')

# 方形波のフーリエ級数展開

${\rm SquareWave}(t)$ は周期 $T$ の周期信号なので，
$$
{\rm SquareWave}(t) = \sum_{k=-\infty}^{\infty} a_ke^{jk\omega_0t} \ , \quad ただし \quad a_k = \frac{1}{T} \int_0^T x(t)e^{-jk\omega_0t}dt
$$
のようにフーリエ級数に展開できる．<br/>
ただし，$\omega_0=\frac{2\pi}{T}$ であり，従って $T\omega_0=2\pi$ である．

フーリエ係数 $a_k$ を詳しく計算すると，まず $k=0$ のとき，
$$
a_0 = \frac{1}{T} \int_0^T x(t)e^0 dt = \frac{1}{T} \int_0^\frac{T}{2}dt =  \frac{1}{T} \left(\frac{T}{2}-0 \right) = \frac{1}{2}
$$
である．一方，$k \neq 0$のとき，
$$
\begin{align}
a_k = \frac{1}{T} \int_0^T x(t)e^{-jk\omega_0t}dt &= \frac{1}{T} \int_0^\frac{T}{2} e^{-jk\omega_0t}dt \\
&= \frac{1}{T} \left[ \frac{1}{-jk\omega_0} e^{-jk\omega_0t} \right]_0^\frac{T}{2} \\
&= -\frac{1}{2jk\pi} \{ e^{-jk\pi} - 1\} = \frac{1}{2jk\pi} \{ 1 - e^{-jk\pi}\}
\end{align}
$$
となる．<br/>
ここで，$e^{-jk\pi}$ は $k$ が偶数のとき 1，奇数のとき -1 であるので，結局
$$
a_k = \left\{
  \begin{array}{ll}
  \frac{1}{2jk\pi} \{ 1 - 1 \} = 0 & (k:偶数) \\
  \frac{1}{2jk\pi} \{ 1 - (-1) \} = \frac{1}{jk\pi} & (k:奇数) \\
  \end{array}
\right.
$$
となる．

上記の $a_k$ を冒頭のフーリエ級数展開の式に代入すると，
$$
\begin{align}
{\rm SquareWave}(t) &= \sum_{\!\!\!\!\!\!\!\!k:負の奇数} \;\; \frac{1}{jk\pi}e^{jk\omega_0t} + \frac{1}{2} + \sum_{\!\!\!\!\!\!\!\!k:正の奇数} \;\; \frac{1}{jk\pi}e^{jk\omega_0t} \\
&= \sum_{\!\!\!\!\!\!\!\!k:正の奇数} \;\; \frac{1}{jk\pi} \{ e^{jk\omega_0t} - e^{-jk\omega_0t} \} + \frac{1}{2} \\
&= \sum_{\!\!\!\!\!\!\!\!k:正の奇数} \frac{2}{k\pi} \sin\left(k\omega_0t\right) + \frac{1}{2} \\
&= \frac{1}{2} + \frac{2}{\pi} \sum_{l=0}^\infty \frac{1}{2l+1} \sin\left((2l+1)\omega_0t\right)
\end{align}
$$
が得られる．


## プログラムによる検証

上のフーリエ級数展開式は無限和を含むため，コンピュータでは計算できない．<br/>
そこで，総和をとる際の $l$ に上限 $M$ を定め，
$$
x_M(t) = \frac{1}{2} + \frac{2}{\pi} \sum_{l=0}^M \frac{1}{2l+1} \sin\left((2l+1)\omega_0t\right)
$$
とする．<br/>
$M \rightarrow \infty$ の極限が方形波，すなわち
$$
\lim_{M \rightarrow \infty} x_M(t) = {\rm SquareWave}(t)
$$
である．<br/>
$M$ を大きくするほど $x_M(t)$ は方形波に近づくはずである．<br/>
これを，以下のコードを実行して確かめよう．

In [ ]:
import numpy as np
from numpy import sin
from math import pi
from signal_processing.utils import show_continuous_signal


### 設定項目：ここから ###

# グラフ表示する範囲を指定
t_min = -3
t_max = 3

### 設定項目：ここまで ###


# x_M(t) を関数として定義
def x_M(t):

  T = 1  # T を設定
  omega_0 = 2 * pi / T  # ω0 を設定

  # l == 0 のとき
  l = 0
  A = 1 / (2*l + 1)  # 正弦波の振幅
  omega = (2*l + 1) * omega_0  # 正弦波の角周波数
  val = A * sin(omega * t)  # val: 時刻 t における信号値

  # 1 <= l <= M のとき
  for l in range(1, M+1):
    A = 1 / (2*l + 1)  # 正弦波の振幅
    omega = (2*l + 1) * omega_0  # 正弦波の角周波数
    val += A * sin(omega * t)  # 時刻 t における信号値を更新

  # 2/π をかけ，さらに定数成分を加算する
  val = (2 / pi) * val + 1 / 2

  # 結果を返す
  return val


# 関数 x_M を連続時間信号と解釈し，t = -3～3 [秒] の範囲をグラフ表示
M = 1
show_continuous_signal(x_M, trange=[t_min, t_max], vrange=[-1.5, 2.5], title='x_M(t) where M={0}'.format(M))
M = 3
show_continuous_signal(x_M, trange=[t_min, t_max], vrange=[-1.5, 2.5], title='x_M(t) where M={0}'.format(M))
M = 6
show_continuous_signal(x_M, trange=[t_min, t_max], vrange=[-1.5, 2.5], title='x_M(t) where M={0}'.format(M))
M = 10
show_continuous_signal(x_M, trange=[t_min, t_max], vrange=[-1.5, 2.5], title='x_M(t) where M={0}'.format(M))
M = 100
show_continuous_signal(x_M, trange=[t_min, t_max], vrange=[-1.5, 2.5], title='x_M(t) where M={0}'.format(M))